In [1]:
import train_3Dden_AR_v28_custom_loop
from train_3Dden_AR_v28_custom_loop import train_seg3D
import os

version=28

job_ind=1
lambda_val_ind_mdiff=2


num_lambda=10
lambda_val_ind_chdiff= (job_ind - 1) % num_lambda 
dose_level_ind= (job_ind - 1) / num_lambda 
dose_level_arr=[ 6, 5, 4, 3, 2, 7, 8]
dose_level=dose_level_arr[int(dose_level_ind)]


epochs=2
batch_size_arr=32
num_iter=8


base_folder='/data01/user-storage/y.zezhang/data_for_zezhang_mar29'
learning_folder='den_3d_v'+str(version)
full_learning_folder=base_folder+'/learning/'

full_learning_folder=os.path.join(full_learning_folder, learning_folder)




weights_name= 'wt_v'+str(version)
loss_fn_name ='ls_v'+str(version)

In [5]:
""" @Author: Ashequr Rahman
    @Date: 2022-05-22  
    @Last Modified by: Ashequr Rahman 
    @Last Modified time: 2022-05-22 
    @Modifications: 1)Implemented 3D denoising model (concat and addition models) 
"""
#=============================================================================================================================
## Import Libraries  ##
#=============================================================================================================================
import sys, os, gc
from pathlib import Path
#sys.path.append(os.path.join(os.path.dirname(__file__), 'src'))
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import h5py
import time
import argparse
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()

from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from tensorflow.keras.layers import Input, Conv3D, Conv3DTranspose, MaxPooling3D, Dropout, BatchNormalization, concatenate, Add, Activation, LeakyReLU
from tensorflow.keras.activations import softmax
from tensorflow.keras.initializers import Constant
import os
#import network model
from Den3D_Model_AR_v1_Loop import build_dennet3D, ClearMemory
plt.switch_backend('Agg')
tf.compat.v1.enable_eager_execution()
#tf.config.run_functions_eagerly(True)

 


In [6]:
  lambda_val_arr_chdiff = [0, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 3e-1, 1e0, 5e0, 1e1]
  lambda_val_arr_mdiff = [0, 1e-1, 5e-1, 1e0, 5e0, 1e1, 3e1, 1e2, 5e2, 1e3]
  lambda_val_chdiff = lambda_val_arr_chdiff[lambda_val_ind_chdiff]
  lambda_val_mdiff = lambda_val_arr_mdiff[lambda_val_ind_mdiff]
  print(f'lambda_val_chdiff: {lambda_val_chdiff}')
  print(f'lambda_val_mdiff: {lambda_val_mdiff}')

lambda_val_chdiff: 0
lambda_val_mdiff: 0.5


In [7]:

  data_folder = f'{base_folder}/training_data_sa_wd_fix2_48cube'
  data_folder_prev = f'{base_folder}/training_data_sa_wd'
  mod_data_folder = f'{base_folder}/learning/{learning_folder}'

  # load protocols
  dose_level_max = 1
  num_z_slice = 64
  num_reg = 1
  Nx_in, Ny_in, Nz_in = 48, 48, 48
  Nz_in_prev = 32
  num_input_channels = 1
  num_output_channels = num_reg
  Nx_out, Ny_out, Nz_out = 48, 48, 48
  Nz_out_prev = 32

  input_shape = (Nx_in, Ny_in, Nz_in, num_input_channels)
  input_shape_prev = (Nx_in, Ny_in, Nz_in_prev, num_input_channels)
  loc_shape = (Nx_in, Ny_in, 1)
  input_shape_orig = (Nz_in, Ny_in, Nx_in, num_input_channels)
  input_shape_orig_prev = (Nz_in_prev, Ny_in, Nx_in, num_input_channels)
  output_shape = (Nx_out, Ny_out, Nz_out, num_output_channels)
  output_shape_orig = (Nz_out, Ny_out, Nx_out, num_output_channels)
  output_shape_orig_prev = (Nz_out_prev, Ny_out, Nx_out, num_output_channels)

  num_pat = 184
  num_data_dict = {
        'hl': {
                'start_ind': 0,
                'end_ind': num_pat-1,
              },
        'def': {
                'start_ind': 0,
                'end_ind': num_pat-1,
              }
      }
 

In [8]:
  loc_arr = ['a', 'i']                  #############################################################
  sev_arr = [100, 175, 250]             #############################################################
  ext_arr = [30, 60]                    #############################################################

  remove_indices = [] # 3, 51, def center not in center  # remove indice 1 in test case#[77,90,110,140]
  num_train_hl = num_data_dict['hl']['end_ind'] - num_data_dict['hl']['start_ind'] + 1
  num_train_def = num_data_dict['def']['end_ind'] - num_data_dict['def']['start_ind'] + 1
  num_train = num_train_hl*len(loc_arr)*len(ext_arr) + num_train_def * len(loc_arr)* len(ext_arr) * len(sev_arr)
  X_data = np.zeros((num_train,) + input_shape)
  print("X_data allocated")
  Y_data = np.zeros((num_train,) + output_shape)
  Y_data_loc = np.zeros((num_train,) + loc_shape)
  Y_data_mask = np.zeros((num_train,) + input_shape)
  print("Y_data allocated")


X_data allocated
Y_data allocated


In [13]:

  pat_id_arr_fname = os.listdir("/data01/user-storage/y.zezhang/segementation/data_for_zezhang_mar29/training_data_sa_wd_fix2_48cube")
  #pat_id_arr_fname = f'{base_folder}/train_pat_list_def_mc.txt'
  pat_id_arr = np.loadtxt(pat_id_arr_fname, dtype = 'str', comments="#", delimiter=",", unpack=False)



In [21]:
def my_read_bin(cur_inp_file, data_type, input_shape):
  A = np.fromfile(cur_inp_file, dtype = data_type)
  A[np.isnan(A)] = 0
  A = np.reshape(A, input_shape)
  A = np.transpose(A, [2, 1, 0, 3])
  return A

In [71]:
sc_data = 1e2
j = 0
hd='hl'
ind_pat=0
cur_inp_file = f'{data_folder}/{pat_id_arr[ind_pat]}/{hd}/recon_pat{pat_id_arr[ind_pat]}_d{dose_level}_it{num_iter}_c30o5.img'
cur_label_file = f'{data_folder}/{pat_id_arr[ind_pat]}/{hd}/recon_pat{pat_id_arr[ind_pat]}_d{dose_level_max}_it{num_iter}_c30o5.img'
cur_X = my_read_bin(cur_inp_file, 'float32', input_shape_orig)
cur_Y = my_read_bin(cur_label_file, 'float32', input_shape_orig)
cur_X = (cur_X - np.min(cur_X))/(np.max(cur_X) - np.min(cur_X))*sc_data
cur_Y = (cur_Y - np.min(cur_Y))/(np.max(cur_Y) - np.min(cur_Y))*sc_data

In [72]:
loc=loc_arr[0]
ext=ext_arr[0]

In [73]:
          def_loc_fname = f'{data_folder_prev}/{pat_id_arr[ind_pat]}/def_mask_d{loc}21{ext}.bin'
          cur_mask = my_read_bin(def_loc_fname, 'uint8', input_shape_orig_prev)
          def_loc_fname = f'{data_folder_prev}/{pat_id_arr[ind_pat]}/def_centroid_d{loc}21{ext}_mod.bin'
          cur_loc = np.fromfile(def_loc_fname, dtype = 'float32').astype(int) - 1 + 1 #0 -based / but nn2D has 1 shift

          X_data[j,:, :, :, :] = cur_X
          Y_data[j,:, :, :, :] = cur_Y
          Y_data_mask[j,:, :, -Nz_in_prev//2+Nz_in//2:Nz_in//2+Nz_in_prev//2, :] = cur_mask #8:39
          Y_data_loc[j, cur_loc[1] , cur_loc[0], 0] = 1.0
          j = j + 1

In [63]:
  X_data = X_data[:j, :, :, :, :]
  Y_data = Y_data[:j, :, :, :, :]
  Y_data_loc = Y_data_loc[:j, :]
  Y_data_mask = Y_data_mask[:j, :, :, :, :]

In [83]:
def_loc_fname

'/data01/user-storage/y.zezhang/data_for_zezhang_mar29/training_data_sa_wd/80846029/def_centroid_da2130_mod.bin'

In [84]:
np.fromfile(def_loc_fname, dtype = 'float32').astype(int)

array([29, 18, 17])

In [74]:
Y_data_loc[j, cur_loc[1] , cur_loc[0], 0]

1.0

In [77]:
Y_data_loc = Y_data_loc[:j, :]

In [82]:
np.max(Y_data_loc)

1.0